## Data Cleaning
The purpose of this notebook is to adequately clean my output of scraped properties into a workable data frame that I can analyse. My reasoning for each decisions is explained per block of code below. 

The following two codeblocks import the necessary modules and read the csv file, also displaying some features to confirm the file is correctly read.

In [1]:
# Import modules
import numpy as np
import pandas as pd


In [2]:
# Load csv & display some basic details
properties = pd.read_csv('properties.csv', header=0)
#display(properties.head())
#display(properties.tail())
display(properties.shape)
#display(properties.columns.tolist())
properties.head()

(11992, 22)

,id,type of property,subtype of property,locality,province,postalCode,price,type of sale,number of bedrooms,living area,...,open fire,terrace,terrace area,garden,garden area,total property area,total land area,number of facades,swimming pool,state of the building
0,10679324,HOUSE,HOUSE,Seraing,Liège,4100,159000,residential_sale,3,14,...,False,True,64.0,True,106.0,87.0,145.0,2.0,False,just_renovated
1,10679325,HOUSE,HOUSE,Vise,Liège,4600,229000,residential_sale,3,19,...,False,True,136.0,NaN,NaN,145.0,184.0,3.0,False,good
2,10679351,HOUSE,VILLA,Vilvoorde,Flemish Brabant,1800,430000,residential_sale,3,17,...,False,True,40.0,True,400.0,132.0,623.0,3.0,False,good
3,10679090,HOUSE,HOUSE,Oudenaarde,East Flanders,9700,439000,residential_sale,2,21,...,False,True,25.0,True,420.0,161.0,680.0,4.0,False,as_new
4,10678835,HOUSE,HOUSE,Dilbeek,Flemish Brabant,1700,529000,residential_sale,4,29,...,False,True,NaN,NaN,NaN,315.0,8002.0,3.0,False,to_renovate


I setup this loop so I could check the value counts for each column and see which data would need to be subsequently cleaned.

Some issues that I can see in the output are:
* A significant quantity of data is missing for some columns, output as 0 values, '0' strings, or NaNs
  * I can detect the presence of NaNs because the total output of each column doesn't always sum to 11992
* The numerical data across the dataframe is a mix of integers and floats
* The 'type of sale' column contains the same output on every row
* The 'fully equipped kitchen' and 'state of the building' columns contain a lot of similarly descriptive strings that could be condensed
* I will need to choose for each column how to treat the NaN data and what assumptions I can make



In [3]:
# Check distribution of unique values in each column
for column in properties.columns:
    column_data = properties[column]
    value_counts = column_data.value_counts()
    print(value_counts , '\n')

id
10679324    1
10524448    1
10461691    1
10638237    1
10356861    1
           ..
10679187    1
10679188    1
10679410    1
10678647    1
10565540    1
Name: count, Length: 11992, dtype: int64 

type of property
APARTMENT    5997
HOUSE        5995
Name: count, dtype: int64 

subtype of property
APARTMENT               4548
HOUSE                   4088
VILLA                    801
DUPLEX                   427
APARTMENT_BLOCK          314
PENTHOUSE                308
GROUND_FLOOR             268
MIXED_USE_BUILDING       240
FLAT_STUDIO              228
EXCEPTIONAL_PROPERTY     150
MANSION                  129
LOFT                      86
COUNTRY_COTTAGE           76
SERVICE_FLAT              74
TOWN_HOUSE                73
BUNGALOW                  47
TRIPLEX                   40
MANOR_HOUSE               22
FARMHOUSE                 20
KOT                       18
CHALET                    14
OTHER_PROPERTY            11
CASTLE                    10
Name: count, dtype: int64 

loca

I can confirm that there are no duplicated rows by using the duplicated and any methods on my dataframe to make a new dataframe that labels every unique row as false, and any subsequent duplicates as true.

In [4]:
# Confirm there are no duplicated rows
duplicates = properties['id'].duplicated().any()
if duplicates:
    print("Some rows are duplicated")
else:
    print("Every row is unique")

Every row is unique


All True and False booleans are converted to binary for easier data manipulation.

In [5]:
# Iterate through rows, then columns and replace booleans with binary
for ind , row in properties.iterrows():
    for column in properties.columns:   
        if row[column] == False:
            properties.at[ind , column] = 0
        elif row[column] == True:
            properties.at[ind , column] = 1

The 'swimming pool' column's empty values are replaced with 0s. 

Since I think that a swimming pool is a **big** selling point for a house, it would be fair to assume that any property that doesn't have a swimming pool listed wouldn't be likely to have a swimming pool at all.

In [6]:
# If NaN is inplace, replace with a 0. Assume that if the data is not entered, the property does not have the attribute.           
#properties['swimming pool'].fillna(0, inplace=True)

I have deleted the 'type of sale' column because every single row has the same value, rendering it useless.

In [7]:
# Delete type_of_sale since the entire column's data is the same.
try:
    properties = properties.drop('type of sale', axis=1)
except:
    pass

For kitchen data, I have cleaned any fully installed outputs by replacing them with 1s, I have replaced '0' strings with nan for no data, and I have replaced anything else with 0s since the kitchen is not *fully* equipped.

In [8]:
# Clean kitchen data
for index, row in properties.iterrows():
        if row['fully equipped kitchen'] == 'usa_installed' or row['fully equipped kitchen'] == 'installed' or row['fully equipped kitchen'] == 'hyper_equipped' or row['fully equipped kitchen'] == 'usa_hyper_equipped': 
            properties.at[index , 'fully equipped kitchen'] = 1
        elif row['fully equipped kitchen'] == '0':
            properties.at[index , 'fully equipped kitchen'] = np.nan            
        elif row['fully equipped kitchen'] != np.nan and row['fully equipped kitchen'] != 1:
            properties.at[index , 'fully equipped kitchen'] = 0

For renovation data I have replaced any unrenovated status with 'needs renovating' and the binary 0 values with nan for no data. The other renovation statuses have stayed the same.

In [9]:
# Clean renovation data
for index, row in properties.iterrows():
    if row['state of the building'] == 'to_be_done_up' or row['state of the building'] == 'to_renovate' or row['state of the building'] == 'to_restore':
        properties.at[index , 'state of the building'] = 'needs renovating'
    elif row['state of the building'] == 0:
        properties.at[index , 'state of the building'] = np.nan
    else:
        pass

I chose to swap the province and locality columns position because it made more sense to me to read the province then locality from left to right.

In [10]:
# Swap province and locality columns
prop_colummns = properties.columns.tolist()
prop_colummns[3] , prop_colummns[4] = prop_colummns[4] , prop_colummns[3]
properties = properties[prop_colummns]

I chose to sort the dataframe by postcode so the dataframe was neater visually.

In [11]:
# Sort rows by price
properties = properties.sort_values(by='price')

I have then filled the empty data in the dataframe 'x' and changed the numeric column types to integers. To do this I had to convert the na values to an unused integer to avoid string errors while changing the column types to int, then change the -1 to an 'x'.

In [12]:
# Replace nan values with -1
#properties = properties.fillna('-1')

# Change numberic column types to int. Use try and except to ignore repeat runs of code.
try:
    for column in properties.columns[5:20]:
        properties[column] = properties[column].astype("float")
except:
    pass
"""
# Return -1 to 'x'
for column in properties.columns:
    for index, value in properties[column].items():
        if value == -1:
            properties.at[index, column] = 'x'
properties
"""

"\n# Return -1 to 'x'\nfor column in properties.columns:\n    for index, value in properties[column].items():\n        if value == -1:\n            properties.at[index, column] = 'x'\nproperties\n"

I run this code to quickly scan for any newly unexpected values, and I see a -1 in provinces

In [13]:
# Check distribution of unique values in each column
for column in properties.columns:
    column_data = properties[column]
    value_counts = column_data.value_counts()
    print(value_counts , '\n')

id
10492614    1
10613566    1
10666499    1
10604617    1
10675026    1
           ..
10436126    1
10199419    1
10434408    1
10483553    1
10303949    1
Name: count, Length: 11992, dtype: int64 

type of property
APARTMENT    5997
HOUSE        5995
Name: count, dtype: int64 

subtype of property
APARTMENT               4548
HOUSE                   4088
VILLA                    801
DUPLEX                   427
APARTMENT_BLOCK          314
PENTHOUSE                308
GROUND_FLOOR             268
MIXED_USE_BUILDING       240
FLAT_STUDIO              228
EXCEPTIONAL_PROPERTY     150
MANSION                  129
LOFT                      86
COUNTRY_COTTAGE           76
SERVICE_FLAT              74
TOWN_HOUSE                73
BUNGALOW                  47
TRIPLEX                   40
MANOR_HOUSE               22
FARMHOUSE                 20
KOT                       18
CHALET                    14
OTHER_PROPERTY            11
CASTLE                    10
Name: count, dtype: int64 

prov

I can run the below code to check whether anything in my data frame is -1 or np.nan

In [14]:
count_neg1 = properties[properties.eq(-1).any(axis=1)].shape[0]
count_nan = properties[properties.eq(np.nan).any(axis=1)].shape[0]
print(count_neg1, count_nan)

0 0


Since there is no -1, I assume the -1 is a string, and I run the below code to prune it, then run the check once more.

In [15]:
for column in properties.columns:
    for index, value in properties[column].items():
        if value == '-1':
            properties.at[index, column] = np.nan
            
count_neg1 = properties[properties.eq('-1').any(axis=1)].shape[0]
count_nan = properties[properties.eq(np.nan).any(axis=1)].shape[0]
print(count_neg1, count_nan)

0 0


I see that the -1 has been cleaned so my data set is now ready for exporting, and use!

In [16]:
properties.to_csv('cleaned_output_x.csv', index=False)